# Sythetic Images

Create images using the fitting parameters. 

### Imports

In [ ]:
# imports
import os, os.path
from dotenv import load_dotenv
import io_files as io
import numpy as np
import json

### Load Setting

In [ ]:
# load default dictionary to put the values
param = {}
# set the settings file path
load_dotenv()
param["s_name"] = os.getenv("S_NAME")
param["s_folder"] = os.getenv("S_FOLDER")
# read the content
path_name = os.path.join(param["s_folder"], param["s_name"])
with open(path_name, "r") as f:
    param.update(json.load(f))
print(param)

### Load results

Load data from npz files

In [ ]:
out = io.Text_Output()
Num = 6

def load_result(d_path):
    result = []
    if os.path.exists(d_path):
        results = np.load(d_path, mmap_mode="r")
        out.showMessage(list(key for key in results.keys()))
        result = [results[key] for key in results.keys()]
    else:
        out.showMessage(f'File: {d_path} not found')
    return result


# set if there are two flats
param["flat2"] = True
# flat1
result1 = load_result(os.path.join(param["output_dir"], param["result_dir"], "flat_fit.npz"))
# image, adjust name of image file
result2 = load_result(os.path.join(param["output_dir"], param["result_dir"], f"image_fit_{Num}.npz"))
# flat2 if present
result3 = []
if param["flat2"]:
    result3 = load_result(os.path.join(param["output_dir"], param["result_dir"], "flat2_fit.npz"))
    # combine results1 and results3
    result4 = list(map(lambda x, y: (x + y) / 2, result1, result3))
else:
    result4 = result1


## Create Image

Pick a sigma from the center to calculate. The flat image is used as the reference.

In [ ]:
def e_func(f):
    def g(x):
        return np.exp(f(x))

    return g

factor = 3
dtype = param["dtype"]

xsize, ysize = result1[0].shape
s_image = np.zeros(result2[0].shape, dtype=float)
for X in range(xsize):
    for Y in range(ysize):
        f_a, f_b, f_c = [result4[i][X, Y] for i in range(3, 6)]
        i_a, i_b, i_c = [result2[i][X, Y] for i in range(3, 6)]
        f_ln_fit = np.poly1d([f_c, f_b, f_a])
        i_ln_fit = np.poly1d([i_c, i_b, i_a])
        f_fit = e_func(f_ln_fit)
        i_fit = e_func(i_ln_fit)
        mu = -f_b/2*f_c
        sigma2 = -1.0 / (2.0 * f_c)
        sigma = np.sqrt(abs(sigma2))
        x_value = mu + factor*sigma
        s_image[X,Y] = i_fit(x_value)/f_fit(x_value)

io.mkdir(os.path.join(param["output_dir"], param["result_dir"], f'{factor}sigma'))
io.save_img(os.path.join(param["output_dir"], param["result_dir"], f'{factor}sigma', f'image_synth_{Num}.tif'), s_image, dtype)